In [1]:
from game import controlled_run


pygame 2.0.1 (SDL 2.0.14, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
import numpy as np
from game import DO_NOTHING
from game import JUMP

games_count=0
total_number_of_games =2


import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

model = Sequential()
model.add(Dense(1,input_dim=1,activation='sigmoid'))
model.add(Dense(2,activation='softmax'))
model.compile(Adam(lr=0.1),loss='categorical_crossentropy',metrics=['accuracy'])

x_train = np.array([])
y_train = np.array([])


# flaw in game design, it should be based on whether or not you make it past an opponent
class Wrapper(object):
    
    
    def __init__(self):
        controlled_run(self,0)
        
    def control(self,values):
        global x_train
        global y_train
        
        print(values)
        
        if(values['closest_enemy'] == -1):
            return DO_NOTHING
        
        if(values['old_closest_enemy']!=-1):
            print(values['closest_enemy'])
            if(values['score_increased']==1):
                x_train = np.append(x_train,[values['old_closest_enemy']/1000])
                y_train = np.append(y_train,[values['action']])
                
        prediction=model.predict_classes(np.array([[values['closest_enemy']]])/1000)
            
        
        return prediction
    def reward(self,values):
        print("No reward")
    def gameover(self,score):
        global games_count
        global x_train
        global y_train
        
        print(x_train)
        print(y_train)
        
        if(games_count >0):
            y_train_cat = to_categorical(y_train,num_classes=2)
            model.fit(x_train,y_train_cat,epochs=50, verbose =1,shuffle=1)
            
        games_count+=1
        
        if games_count>total_number_of_games: 
            return
        controlled_run(self,games_count)



In [5]:
w=Wrapper()

No reward
{'action': 0, 'old_closest_enemy': -1, 'closest_enemy': -1, 'score_increased': 1}
No reward
{'action': 0, 'old_closest_enemy': -1, 'closest_enemy': -1, 'score_increased': 1}
No reward
{'action': 0, 'old_closest_enemy': -1, 'closest_enemy': -1, 'score_increased': 1}
No reward
{'action': 0, 'old_closest_enemy': -1, 'closest_enemy': -1, 'score_increased': 1}
No reward
{'action': 0, 'old_closest_enemy': -1, 'closest_enemy': -1, 'score_increased': 1}
No reward
{'action': 0, 'old_closest_enemy': -1, 'closest_enemy': 862, 'score_increased': 1}
/usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it u

In [23]:
from game import controlled_run

import random
import numpy as np
from collections import deque




GAMMA = 0.95
LEARNING_RATE = 0.001

MEMORY_SIZE = 1000000
BATCH_SIZE = 20

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.0
EXPLORATION_DECAY = 0.995

import numpy as np
from game import DO_NOTHING
from game import JUMP

games_count=0
total_number_of_games =50


import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

class DQNSolver:

    def __init__(self, observation_space, action_space):
        self.exploration_rate = EXPLORATION_MAX

        self.action_space = action_space
        self.memory = deque(maxlen=MEMORY_SIZE)

        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        # self.memory.append((gamenumber,state,action,reward,next_state,done))

    def act(self, state):
        if np.random.rand() < self.exploration_rate:
            return random.randrange(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])
    # fit on past data
    def experience_replay(self):
        print("memory: " + str(len(self.memory)))
        if len(self.memory) < BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        # gamenumber = random(0,10)
        # self.memory(gamenumber)
        cul_reward = 0
        index=0
        lastmove = len(batch)
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            cul_reward+=reward
            if not terminal:
                
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
            q_values = self.model.predict(state)
            #print(np.argmax(q_values[0]))
            #update the q value for that action 
            q_values[0][action] = q_update
            #Fitting during experience replay?
            self.model.fit(np.array(state), q_values, verbose=0)
            index+=1
        decay = cul_reward/500

        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)

In [40]:


# flaw in game design, it should be based on whether or not you make it past an opponent
class Wrapper(object):
    
    
    def __init__(self,dqn_solver):
        self.dqn_solver = dqn_solver
        controlled_run(self,0)
        
    def control(self,values):
        dqn_solver = self.dqn_solver
        state = [[values['closest_enemy']/1000]]
        step = 0 

        self.old_state = state
        action = dqn_solver.act(state)
        self.action = action
        
        return action
        
    def reward(self,values):
        
        dqn_solver = self.dqn_solver

        state_next = values['closest_enemy']
        reward= values['score_increased']
        terminal=False
        try:
            self.state = state_next
            dqn_solver.remember(self.old_state, self.action, reward, [state_next], terminal)
            print("remembered")
        except:
            pass
    
    def gameover(self,score):
        global games_count
        dqn_solver = self.dqn_solver
        reward = -1
        dqn_solver.remember(self.old_state, self.action, reward, self.state, True)
        try:
            print("Run: " + str(games_count) + ", exploration: " + str(dqn_solver.exploration_rate) + ", score: " + str(score))
        except:
            print("Run: " + str(games_count) + ", score: " + str(score))

        if(games_count >0):
            print("replaying...")
            dqn_solver.experience_replay()

        games_count+=1
        
        if games_count>total_number_of_games: 
            return
        controlled_run(self,games_count)

In [52]:
gamma = 0.95
alpha = 0.5
learning_rate_adam = 0.01
epsilon = 0.999
epsilon_decay = 0.99
class DQN:

    def __init__(self, observation_space, action_space):
        
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma
        self.action_space = action_space
        self.observation_space = observation_space
        
        self.memory = []
        self.batch_size = 20

        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(self.observation_space,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=learning_rate_adam))

    def act(self, state):
        if np.random.rand() < self.epsilon:
            return random.randrange(self.action_space)
        q = self.model.predict(state)
        return np.argmax(q[0])

    def experience_replay(self):
        print(self.memory,self.batch_size)
        mem_sample = random.sample(self.memory, min(len(self.memory),self.batch_size))
        for state, action, reward, next_state, done in mem_sample:
            update_value = reward
            if not done:
                update_value = self.alpha * (reward + self.gamma * np.max(self.model.predict(next_state)[0]))
            q = self.model.predict(state)
            q[0][action] = update_value
            # print(state,type(state))
            # print(q,type(q))
            self.model.fit(np.array(state), q, verbose=0)
        self.epsilon *= epsilon_decay

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

In [51]:
dqn_solver = DQN(1,2)
game = Wrapper(dqn_solver)

/usr/local/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
remembered
Run: 19, score: 7
replaying...
[([[-0.001]], 0, 1, [-1], False), ([[-0.001]], 0, 1, [-1], False), ([[-0.001]], 0, 1, [-1], False), ([[-0.001]], 1, 0, [-1], False), ([[-0.001]], 0, 1, [-1], False), ([[-0.001]], 1, 0, [-1], False), ([[-0.001]], 0, 1, [994], False), ([[0.994]], 0, 1, [838], False), ([[0.838]], 1, 0, [529], False), ([[0.529]], 1, 0, [220], False), ([[0.22]], 0, -1, 220, True)] 20
[[-0.001]] <class 'list'>
[[6.583896e-01 3.334520e-04]] <class 'numpy.ndarray'>


ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {"<class \'float\'>"})'}), <class 'numpy.ndarray'>